<a href="https://colab.research.google.com/github/jibintom/Machine-Learning-Codebasics-/blob/main/a20.%20Ensemble%20Learning%20-%20Bagging/Bagging_Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ensemble Learning: Bagging**

---



Download heart disease dataset heart.csv in Exercise folder and do following, (credits of dataset: https://www.kaggle.com/fedesoriano/heart-failure-prediction)

1. Load heart disease dataset in pandas dataframe

2. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
3. Convert text columns to numbers using label encoding and one hot encoding
4. Apply scaling
5. Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
6. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
7. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best.

In [40]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

df=pd.read_csv("heart_21.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [41]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [42]:
df.shape

(918, 12)

In [43]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


**Treat Outliers**

In [44]:
df[(df.Cholesterol> (df.Cholesterol.mean()+3*df.Cholesterol.std())) | (df.Cholesterol< (df.Cholesterol.mean()-3*df.Cholesterol.std()))]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
76,32,M,ASY,118,529,0,Normal,130,N,0.0,Flat,1
149,54,M,ASY,130,603,1,Normal,125,Y,1.0,Flat,1
616,67,F,NAP,115,564,0,LVH,160,N,1.6,Flat,0


In [45]:
df1=df[(df.Cholesterol< (df.Cholesterol.mean()+3*df.Cholesterol.std())) & (df.Cholesterol > (df.Cholesterol.mean()-3*df.Cholesterol.std()))]
df1.shape

(915, 12)

In [46]:
df1[(df1.RestingBP> (df1.RestingBP.mean()+3*df1.RestingBP.std())) | (df1.RestingBP< (df1.RestingBP.mean()-3*df1.RestingBP.std()))]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
109,39,M,ATA,190,241,0,Normal,106,N,0.0,Up,0
241,54,M,ASY,200,198,0,Normal,142,Y,2.0,Flat,1
365,64,F,ASY,200,0,0,Normal,140,Y,1.0,Flat,1
399,61,M,NAP,200,0,1,ST,70,N,0.0,Flat,1
449,55,M,NAP,0,0,0,Normal,155,N,1.5,Flat,1
592,61,M,ASY,190,287,1,LVH,150,Y,2.0,Down,1
732,56,F,ASY,200,288,1,LVH,133,Y,4.0,Down,1
759,54,M,ATA,192,283,0,LVH,195,N,0.0,Up,1


In [47]:
df2=df1[(df1.RestingBP < (df1.RestingBP.mean()+3*df1.RestingBP.std())) & (df1.RestingBP > (df1.RestingBP.mean()-3*df1.RestingBP.std()))]
df2.shape

(907, 12)

In [48]:
df2[(df2.MaxHR> (df2.MaxHR.mean()+3*df2.MaxHR.std())) | (df2.MaxHR< (df2.MaxHR.mean()-3*df2.MaxHR.std()))]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
390,51,M,ASY,140,0,0,Normal,60,N,0.0,Flat,1


In [49]:
df3=df2[(df2.MaxHR < (df2.MaxHR.mean()+3*df2.MaxHR.std())) & (df2.MaxHR > (df2.MaxHR.mean()-3*df2.MaxHR.std()))]
df3.shape

(906, 12)

In [50]:
df3[(df3.Oldpeak	> (df3.Oldpeak	.mean()+3*df3.Oldpeak	.std())) | (df3.Oldpeak	< (df3.Oldpeak	.mean()-3*df3.Oldpeak	.std()))]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
166,50,M,ASY,140,231,0,ST,140,Y,5.0,Flat,1
324,46,M,ASY,100,0,1,ST,133,N,-2.6,Flat,1
702,59,M,TA,178,270,0,LVH,145,N,4.2,Down,0
771,55,M,ASY,140,217,0,Normal,111,Y,5.6,Down,1
791,51,M,ASY,140,298,0,Normal,122,Y,4.2,Flat,1
850,62,F,ASY,160,164,0,LVH,145,N,6.2,Down,1
900,58,M,ASY,114,318,0,ST,140,N,4.4,Down,1


In [51]:
df4=df3[(df3.Oldpeak	 < (df3.Oldpeak	.mean()+3*df3.Oldpeak	.std())) & (df3.Oldpeak	 > (df3.Oldpeak	.mean()-3*df3.Oldpeak	.std()))]
df4.shape

(899, 12)

**Treating Categorical Values**

In [52]:
df.ChestPainType.unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [53]:
df.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [54]:
df.ExerciseAngina.unique()

array(['N', 'Y'], dtype=object)

In [55]:
df.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [56]:
df5=df4.copy()

df5.ExerciseAngina.replace({'N':0, 'Y':1}, inplace=True)
df5.ST_Slope.replace({'Up':3, 'Flat':2, 'Down':1}, inplace=True)
df5.RestingECG.replace({'Normal':1, 'ST':2, 'LVH':3}, inplace=True)
df5.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,1,172,0,0.0,3,0
1,49,F,NAP,160,180,0,1,156,0,1.0,2,1
2,37,M,ATA,130,283,0,2,98,0,0.0,3,0
3,48,F,ASY,138,214,0,1,108,1,1.5,2,1
4,54,M,NAP,150,195,0,1,122,0,0.0,3,0


In [57]:
df6=pd.get_dummies(df5, drop_first=True)
df6.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,1,172,0,0.0,3,0,1,1,0,0
1,49,160,180,0,1,156,0,1.0,2,1,0,0,1,0
2,37,130,283,0,2,98,0,0.0,3,0,1,1,0,0
3,48,138,214,0,1,108,1,1.5,2,1,0,0,0,0
4,54,150,195,0,1,122,0,0.0,3,0,1,0,1,0


**Scaling**

In [58]:
x=df6.drop("HeartDisease", axis=1)
y=df6.HeartDisease

In [61]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
x_mms=mms.fit_transform(x)
x_mms[:3]

array([[0.24489796, 0.57142857, 0.55791506, 0.        , 0.        ,
        0.78417266, 0.        , 0.33333333, 1.        , 1.        ,
        1.        , 0.        , 0.        ],
       [0.42857143, 0.76190476, 0.34749035, 0.        , 0.        ,
        0.66906475, 0.        , 0.5       , 0.5       , 0.        ,
        0.        , 1.        , 0.        ],
       [0.18367347, 0.47619048, 0.54633205, 0.        , 0.5       ,
        0.25179856, 0.        , 0.33333333, 1.        , 1.        ,
        1.        , 0.        , 0.        ]])

In [66]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

**SVM using standalone model**

In [62]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

scores=cross_val_score(SVC(), x,y, cv=5)
scores

array([0.62222222, 0.79444444, 0.69444444, 0.72777778, 0.62011173])

In [63]:
scores.mean()

0.6918001241464928

**SVM using Bagging**

In [67]:
from sklearn.ensemble import BaggingClassifier 
model=BaggingClassifier(
                        base_estimator=SVC(),
                        n_estimators=100, 
                        random_state=0,
                        oob_score=True,
                        max_samples=.8).fit(x_test,y_test)

model.oob_score_

0.6902356902356902

In [68]:
model.score(x_test,y_test)

0.7070707070707071

As you can see above, using bagging in case of SVM doesn't make much difference in terms of model accuracy. Bagging is effective when we have high variance and instable model such as decision tree. Let's explore how bagging changes the performance for a decision tree classifier.

**decision tree classifier using standalone model**

In [72]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

scores=cross_val_score(DecisionTreeClassifier(), x,y, cv=5)
scores

array([0.74444444, 0.8       , 0.76111111, 0.68333333, 0.59776536])

In [73]:
scores.mean()

0.7173308504034761

**decision tree classifier using Bagging**

In [76]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier 
model=BaggingClassifier(
                        base_estimator=DecisionTreeClassifier(),
                        n_estimators=100, 
                        random_state=0,
                        oob_score=True,
                        max_samples=.8).fit(x,y)

model.oob_score_

0.8476084538375973

**You can see that with bagging the score improved from 71.93% to 84.37%**